## Call Tools to provide more accurate responses

In [1]:
from langchain.agents import Tool
from langchain.agents import tool
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

In [2]:
@tool
def add_number(inputs: str) :
    """
    Tool: add_numbers
    Description: Add numeric values extracted from text input
    Input: str (e.g. "Add numbers 10, 20 and 30")
    Output: dict (e.g. {"result": 60})
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    result = sum(numbers)
    return str(result)

In [ ]:
# add_tool = Tool(
#     name="AddTool",
#     func=add_number,
#     description="Add a list of numbers and return the result",
# )
# print("Tool Object:", add_tool)


In [3]:
test_input = "What is the sum of 10, 20 and 30"
print(add_number.invoke(test_input))

test_input = "What is the sum of ten, 20 and 30"
print(add_number.invoke(test_input))

60
50


In [4]:
model = init_chat_model("openai:gpt-4.1", temperature=0)
agent = create_react_agent(
    tools=[add_number],
    model=model,
)


In [5]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the sum of 10, 20 and 30"}]}
)
for message in result["messages"]:
    print(message.content)


What is the sum of 10, 20 and 30

60
The sum of 10, 20, and 30 is 60.


In [6]:
from functools import reduce


def multiply_number(inputs: str) :
    """
    Tool: multiply_numbers
    Description: Multiply numeric values extracted from text input
    Input: str (e.g. "Multiply numbers 10, 20 and 30")
    Output: dict (e.g. {"result": 60})
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    if not numbers:
        return "No numbers found in input"
    result = reduce(lambda x, y: x * y, numbers)
    return str(result)

def divide_number(inputs: str) :
    """
    Tool: divide_numbers
    Description: Divide numeric values extracted from text input
    Input: str (e.g. "Divide numbers 10, 20 and 30")
    Output: dict (e.g. {"result": 60})
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    if not numbers:
        return "No numbers found in input"
    result = reduce(lambda x, y: x / y, numbers)
    return str(result)

In [7]:
model = init_chat_model("openai:gpt-4.1", temperature=0)
agent = create_react_agent(
    tools=[add_number, multiply_number, divide_number],
    model=model,
    prompt= "You are a helpful math assistant that can perform addition, multiplication and division. Use the tools to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Also show your reasoning step by step.",
)
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the sum of 10, 20 and 30 multiplied by 2 divided by 3?"}]}
)
for message in result["messages"]:
    print(message.content)


What is the sum of 10, 20 and 30 multiplied by 2 divided by 3?

60

120

40.0
First, add 10, 20, and 30 to get 60. Then, multiply 60 by 2 to get 120. Finally, divide 120 by 3 to get 40.

So, the answer is 40.


In [8]:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun, WikipediaAPIWrapper
from langchain_community.tools.google_finance.tool import GoogleFinanceQueryRun, GoogleFinanceAPIWrapper
from langchain_community.tools.golden_query.tool import GoldenQueryRun, GoldenQueryAPIWrapper
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
google_finance_tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())
golden_query_tool = GoldenQueryRun(api_wrapper=GoldenQueryAPIWrapper())
model = init_chat_model("openai:gpt-4.1", temperature=0)
agent = create_react_agent(
    tools=[add_number, multiply_number, divide_number, wiki_tool, google_finance_tool, golden_query_tool],
    model=model,
    prompt= "You are a helpful assistant that can perform various tasks. Use the tools to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Also show your reasoning step by step, and cite any sources your used"
)

In [9]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is Bell's inequality"}]}
)
for message in result["messages"]:
    print(message.content)

What is Bell's inequality

Page: Bell's theorem
Summary: Bell's theorem is a term encompassing a number of closely related results in physics, all of which determine that quantum mechanics is incompatible with local hidden-variable theories, given some basic assumptions about the nature of measurement. The first such result was introduced by John Stewart Bell in 1964, building upon the Einstein–Podolsky–Rosen paradox, which had called attention to the phenomenon of quantum entanglement.
In the context of Bell's theorem, "local" refers to the principle of locality, the idea that a particle can only be influenced by its immediate surroundings, and that interactions mediated by physical fields cannot propagate faster than the speed of light. "Hidden variables" are supposed properties of quantum particles that are not included in quantum theory but nevertheless affect the outcome of experiments. In the words of Bell, "If [a hidden-variable theory] is local it will not agree with quantum me